<h1 style="color:blue;">Overview 3: Concatenation and Merging of Tables</h1>

One of the main advantages of SAS is the ability to combine multiple tables to create a new table.  
Two possibilities are offered: **concatenation** and **merging**.

## 1. Concatenation

Concatenation involves "vertically stacking" the observations from two tables, `tab1` and `tab2`, so that only one table is produced. This is done using the `Set` statement.  

**Example:** Consider the following two tables, `tab1` and `tab2`:

### Tab1
| Name  | Score |
|-------|-------|
| toto  | 10    |
| titi  | 09    |
| tata  | 15    |

### Tab2
| Name  | Score |
|-------|-------|
| titi  | 12    |
| tutu  | 13    |

The following program:  

In [ ]:
Data tab3; set Tab1 Tab2;
Run;

Results in the following table:

**Tab3**
| Name  | Score |
|-------|-------|
| toto  | 10    |
| titi  | 09    |
| tata  | 15    |
| titi  | 12    |
| tutu  | 13    |


- Columns with the same name are merged into a single column.  
- The individuals in `Tab2` are appended to those in `Tab1` without sorting.

### Interleaving of Two (or More) Tables

In the previous example, the name "Titi" is positioned according to its order of appearance in both `Tab1` and `Tab2`.

**Alternative:** Names can be ordered alphabetically in `Tab3`. To do this:


In [ ]:
Proc sort data=Tab1;
By name;
Run;

Proc sort data =Tab2;
By name;
Run;

Data Tab3; 
Set Tab1 Tab2;
By name;
Run;

**Important:**  
For interleaving of tables, they need to be sorted first using the `Proc Sort` statement, with a defined sorting key (`By` statement).

The same result can be achieved with a simple concatenation followed by sorting on the newly created table:


In [ ]:
Data Tab3; Set Tab1 Tab2;
Run;

Proc sort data=Tab3;
By name;
Run;

**Distinction Based on Data Source**  
  - Suppose the scores from the two tables have different meanings.   
  - You can create two different variables for "score" as follows:   

In [ ]:
Data Tab3;
Set Tab1 Tab2(Rename =(score=score1));
Run;

Tab3
| Name  | Score | Score1 |
|-------|-------|--------|
| toto  | 10    |    .   |
| titi  | 09    |    .   |
| tata  | 15    |    .   |
| tata  |  .    |   12   |
| tata  |   .   |   13   |


**Note the importance** of the placement of options like Drop, Keep, and Rename between the Data or Set statement.

The drawback of this method is the creation of missing values.
Another approach is to create a variable to identify the source of the observations.


In [ ]:
Data Tab3; Set Tab1(in=ori);
If ori =1 then source="a"; Else source="b";
Run;

**Tab3**   
| Name | Score | Source |
|------|-------|--------|
| toto | 10    | a      |
| titi | 09    | a      |
| tata | 15    | a      |
| titi | 12    | b      |
| tutu | 13    | b      |


**2. Merging Multiple Tables**
Often, we want to combine information for the same individual (statistically speaking) that is spread across multiple tables.
To do this, we use the Merge statement, always paired with a By statement based on the individual for merging.

Consider the following tables:

 **Tab1**
| Company  | Revenue | NACE |
|----------|---------|------|
| Valeo    | 100     | 32   |
| Peugeot  | 760     | 33   |
| Airbus   | 1500    | 41   |
| Vallourec| 350     | 53   |

 **Tab02**
| Company  | Profit |
|----------|--------|
| Valeo    | -12    |
| Peugeot  | -13    |
| Airbus   | 174    |
| Vallourec| 27     |

 

The goal is to merge Tab1 and Tab2 so that we have all information (Revenue, Profit, and NACE code) for each company in one table (Tab3) for the year 2009.

In [ ]:
Proc sort data=tab1;
By Company;
Run;
Proc sort data=tab2;
By Company;
Run;
Data Tab3;
Merge Tab1 Tab2;
By Company;
Run;


**Tab03**
| Company  | Revenue | NACE | Profit |
|----------|---------|------|--------|
| Airbus   | 1500    | 41   | 174    |
| Bouygues | 1235    | 21   | .      |
| Peugeot  | 760     | 33   | -13    |
| Valeo    | 100     | 32   | -12    |
| Vallourec| 350     | 53   | 27     |


If the two tables have different numbers of individuals, the new table will have as many rows/individuals as the larger of the two tables.
To fix this, we use:

In [ ]:
Data Tab3;
Merge Tab1(in=in1) Tab2(in=in2);
By Company;
If in1=1 and in2=1;
Run;


This ensures that only companies present in both tables will be included.

**Tab3**
| Company  | Revenue | NACE | Profit |
|----------|---------|------|--------|
| Airbus   | 1500    | 41   | 174    |
| Peugeot  | 760     | 33   | -13    |
| Valeo    | 100     | 32   | -12    |
| Vallourec| 350     | 53   | 27     |


What Happens if the Merge is Done Without By?
If Tab1 and Tab2 have the same number of rows, and you rename the Company column in one of the tables, the result will be a horizontal concatenation.

**Tab03**
| Company  | Revenue | NACE | Company_1 | Profit |
|----------|---------|------|-----------|--------|
| Valeo    | 100     | 32   | Valeo     | -12    |
| Peugeot  | 760     | 33   | Peugeot   | -13    |
| Airbus   | 1500    | 41   | Airbus    | 174    |
| Vallourec| 350     | 53   | Vallourec | 27     |


---